<h1 style="text-align: center;">Emotion Recognition Inference with Arduino Nano 33 BLE</h1>

## Introduction
In this notebook, we demonstrate the deployment of the speech emotion recognition (SER) model, previously developed and converted into a TinyML model in [this notebook], with an Arduino Nano 33 BLE microcontroller, a compact and energy-efficient board that is ideal for implementing real-time applications. The primary objective of this notebook is to showcase the process of processing audio data from a microphone connected to the board and running inferences on the TensorFlow Lite (TFLite) model designed to classify four emotions: Happy, Surprised, Neutral, and Unpleasant.

We begin by setting up a serial connection with the Arduino Nano 33 BLE and calibrating the microphone for reliable data collection. Next, we define asynchronous functions for processing the data and running the model. We also measure the model's inference time to evaluate its performance on the microcontroller.

By the end of this notebook, you will have a better understanding of how to implement an SER model with a resource-constrained device, opening the door to applications in the fields of affective computing and IoT.

## Setting Up
1. Connect your Arduino 33 BLE Sense to your computer using a USB cable.
2. Open the Arduino IDE on your computer and select the correct board and port under the "Tools" menu.
3. Upload the [Data Streaming Code] provided in the project repository to your Arduino board.
4. After uploading, open the Serial Monitor in the Arduino IDE and make sure the baud rate is set to 38400.
5. Press the reset button on the Arduino board to initialize it with the greeting message "Welcome to the data streaming...". If you see this message, congratulations! You can move on to the next step.

> **Note:** Sometimes the board may go to sleep. If you experience any issues, press the reset button on the Arduino board again before running the code below.

## Importing Libraries
Include the necessary libraries for audio processing, communication with the Arduino, and TinyML inference in real time:

In [1]:
import asyncio
import time
from serial.tools import list_ports

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import serial
from IPython.display import Audio, display, HTML
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras


ports = list_ports.comports()
for port in ports:
    print(port)

2023-03-30 17:30:48.795792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/dev/cu.BLTH - n/a
/dev/cu.Bluetooth-Incoming-Port - n/a
/dev/cu.usbmodem14401 - Nano 33 BLE


## Initialization
In this section, the serial connection with the Arduino Nano 33 BLE is established, and the microphone is calibrated to normalize the input data within a specific range for improved processing and feature extraction.

In [2]:
SERIAL_PORT = '/dev/cu.usbmodem14401'
BAUD_RATE = 38400

def arduino_activate():
    try:
        arduino = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
        time.sleep(2)  # Allow time for connection to establish
    except serial.SerialException as e:
        print(f"Failed to connect to serial port {SERIAL_PORT}: \n{e}")
        return None

    command = input("Type 'x' to Activate Arduino: ")
    if command.lower() == 'x':
        arduino.write(command.encode('utf-8'))
        print("Success Activated!")
        print("Open the Serial Monitor to check if it is working, then close it.")
    return arduino


arduino = arduino_activate()

Type 'x' to Activate Arduino: x
Success Activated!
Open the Serial Monitor to check if it is working, then close it.


In [3]:
def arduino_read(buffer, buffer_size, overlapping, norm=(None, None), max_attempts=10):
    buffer = np.roll(buffer, overlapping)
    num_data = buffer_size - overlapping
    for i in range(num_data):
        decoded_data = ''
        attempts = 0
        while decoded_data == '':
            arduino_data = arduino.readline()
            decoded_data = arduino_data[:len(arduino_data)].decode("utf-8").strip('\r\n')
            attempts += 1
            if attempts >= max_attempts:
                print('Fail to Retrieve Data...')
                break
        if norm[0] is not None:
            decoded_data = normalize(int(decoded_data), 1, -1, norm[0], norm[1])
        try:
            buffer[i+overlapping] = decoded_data
        except ValueError:
            print("Open the Serial Monitor to check if it is working. If it's not, press the reset button and rerun 'arduino_activate' funciton again.")
    return buffer

def normalize(array, new_max, new_min, old_max, old_min):
    array = (((array - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
    return array.astype(np.float16)


# Calibrate the microphone
print("Calibrating: Please Speak to the Microphone")
time.sleep(1)
tuning_data = arduino_read(np.zeros(48000), 48000, 0)
TUNING_MAX, TUNING_MIN = (max(tuning_data), min(tuning_data))
print(f'Normalized signal from the range ({TUNING_MIN}, {TUNING_MAX}) to (-1, 1)')

Calibrating: Please Speak to the Microphone
Normalized signal from the range (-553.0, 839.0) to (-1, 1)


## Running Inference
In this section, we set up asynchronous functions for MFCC features extraction. Then, we initialize the TensorFlow Lite interpreter, run emotion recognition inference, and send results back to the Arduino Nano 33 BLE.

In [9]:
BUFFER_SIZE = 24000
OVERLAPPED = 512

NUM_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512
SAMPLE_RATE = 16000

EMOTIONS = ['neutral', 'happy', 'surprise', 'unpleasant']
COMMANDS = ['a', 'b', 'c', 'd']

# Manually Calibrate Sensitivity of Recognition
RECOG_MASK = np.array([1, 30000, 40000, 500])


async def tflite_process_data():
    data = np.zeros(BUFFER_SIZE)
    data = arduino_read(data, BUFFER_SIZE, OVERLAPPED, norm=(TUNING_MAX, TUNING_MIN))
    mfcc = librosa.feature.mfcc(y=data, sr=SAMPLE_RATE, n_mfcc=NUM_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH)
    features = np.array([mfcc.T], dtype=np.float32)

    # Model Input Shape = (None, None, 13)
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], features)
    interpreter.invoke()
    prediction = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])[0]
    result = np.multiply(prediction, RECOG_MASK)
    emotion = EMOTIONS[np.argmax(result)]
    command = COMMANDS[np.argmax(result)]
    arduino.write(command.encode('utf-8'))
    print(result)
    print(f'Emotion: {emotion}\n')
    
async def tflite_run(rounds=10):
    tasks = []
    start_time = time.time()
    for turn in range(rounds):
        task = asyncio.create_task(tflite_process_data())
        tasks.append(task)
        time.sleep(0.6)
    await asyncio.gather(*tasks)
    display(HTML("<hr>"))
    print(f"Inference time for {turn+1} rounds: {time.time() - start_time} seconds") 


interpreter = tf.lite.Interpreter(model_path="output/Models/SER_quant.tflite")
interpreter.allocate_tensors()
await tflite_run(rounds=10)

[7.80900940e-04 3.43083171e+00 1.64198561e+01 4.99347121e+02]
Emotion: unpleasant

[0.99700433 0.8471093  2.38675348 1.45389081]
Emotion: surprise

[0.98819506 1.08457843 8.62430839 5.77661814]
Emotion: surprise

[2.12448905e-03 4.15599687e+00 1.75587903e+01 4.98649031e+02]
Emotion: unpleasant

[9.02240630e-04 4.06786727e+00 1.69497880e+01 4.99269247e+02]
Emotion: unpleasant

[6.84981374e-03 1.01755920e+01 4.90557821e+01 4.95792329e+02]
Emotion: unpleasant

[4.37238440e-03 6.23137486e+00 3.28392955e+01 4.97299433e+02]
Emotion: unpleasant

[5.65976952e-04 2.42617920e+00 1.16034399e+01 4.99531537e+02]
Emotion: unpleasant

[1.48905376e-02 1.65415776e+01 4.07523941e+01 4.91769612e+02]
Emotion: unpleasant

[1.49475290e-02 2.07651086e+01 6.74704323e+01 4.91336733e+02]
Emotion: unpleasant



Inference time for 10 rounds: 13.163029193878174 seconds
